In [16]:
import pandas as pd

# -----------------------------
# Helper functions
# -----------------------------
def cricket_overs_to_balls(overs):
    overs_int = int(overs)
    balls_part = int(round((overs - overs_int) * 10))
    return overs_int * 6 + balls_part

def balls_to_cricket_overs(balls):
    overs = balls // 6
    rem_balls = balls % 6
    return float(f"{int(overs)}.{int(rem_balls)}")

def corrected_actual_overs(row):
    adjusted_ball = row['Actual Ball']
    if row['Legal Ball'] != 'Yes':
        adjusted_ball = max(0, adjusted_ball - 1)
    if adjusted_ball == 6:
        return float(f"{int(row['Over'])}.0")
    else:
        return float(f"{int(row['Over']) - 1}.{int(adjusted_ball)}")

# -----------------------------
# Load and preprocess data
# -----------------------------
data = pd.read_csv("C:/Users/rahul/anaconda_projects/1a85c8cd-a5a1-4bbd-ac45-903f158b07a8/9JuneuptoWT20.csv")
data.columns = data.columns.str.strip()
data[['Team1', 'Team2']] = data['Match'].str.split(' v ', expand=True)
data.sort_values(by=['Match', 'Date', 'Innings'], inplace=True)
data['Next_Innings'] = data.groupby(['Match', 'Date'])['Innings'].shift(-1)
mask = data['Innings'].ne(data['Next_Innings']) & (data.index < len(data))
rows_before_change = data[mask].copy().reset_index(drop=True).drop(columns=['Next_Innings'])

# -----------------------------
# Compute Overs and NRR Balls
# -----------------------------
rows_before_change['Actual Overs'] = rows_before_change.apply(corrected_actual_overs, axis=1)
rows_before_change['NRR Overs'] = rows_before_change.apply(
    lambda row: 20.0 if row['Team Wickets'] == 10 else row['Actual Overs'], axis=1
)
rows_before_change['NRR Balls'] = rows_before_change['NRR Overs'].apply(cricket_overs_to_balls)

# -----------------------------
# Compute Team-wise summaries
# -----------------------------
for_summary = rows_before_change.groupby('Batting Team').agg({
    'Team Runs': 'sum',
    'NRR Balls': 'sum'
}).reset_index().rename(columns={
    'Batting Team': 'Team',
    'Team Runs': 'Runs For',
    'NRR Balls': 'NRR Balls For'
})
for_summary['Overs For'] = for_summary['NRR Balls For'].apply(balls_to_cricket_overs)
for_summary.drop(columns='NRR Balls For', inplace=True)

against_summary = rows_before_change.groupby('Bowling Team').agg({
    'Team Runs': 'sum',
    'NRR Balls': 'sum'
}).reset_index().rename(columns={
    'Bowling Team': 'Team',
    'Team Runs': 'Runs Against',
    'NRR Balls': 'NRR Balls Against'
})
against_summary['Overs Against'] = against_summary['NRR Balls Against'].apply(balls_to_cricket_overs)
against_summary.drop(columns='NRR Balls Against', inplace=True)

team_summary = pd.merge(for_summary, against_summary, on='Team', how='outer')

# -----------------------------
# Calculate NRR
# -----------------------------
team_summary['NRR Balls For'] = team_summary['Overs For'].apply(cricket_overs_to_balls)
team_summary['NRR Balls Against'] = team_summary['Overs Against'].apply(cricket_overs_to_balls)
team_summary['Run Rate For'] = team_summary['Runs For'] / (team_summary['NRR Balls For'] / 6)
team_summary['Run Rate Against'] = team_summary['Runs Against'] / (team_summary['NRR Balls Against'] / 6)
team_summary['NRR'] = (team_summary['Run Rate For'] - team_summary['Run Rate Against']).round(3)
final_nrr_table = team_summary[['Team', 'Runs For', 'Overs For', 'Runs Against', 'Overs Against', 'NRR']]

# -----------------------------
# Compute Match Outcome Stats
# -----------------------------
match_results = []
innings_grouped = rows_before_change.groupby(['Match', 'Date'])

for (match, date), group in innings_grouped:
    if len(group) == 2:
        team1 = group.iloc[0]['Batting Team']
        team2 = group.iloc[1]['Batting Team']
        runs1 = group.iloc[0]['Team Runs']
        runs2 = group.iloc[1]['Team Runs']

        if runs1 > runs2:
            match_results.extend([
                {'Team': team1, 'W': 1, 'L': 0, 'T': 0, 'N/R': 0},
                {'Team': team2, 'W': 0, 'L': 1, 'T': 0, 'N/R': 0},
            ])
        elif runs1 == runs2:
            match_results.extend([
                {'Team': team1, 'W': 0, 'L': 0, 'T': 1, 'N/R': 0},
                {'Team': team2, 'W': 0, 'L': 0, 'T': 1, 'N/R': 0},
            ])
        else:
            match_results.extend([
                {'Team': team1, 'W': 0, 'L': 1, 'T': 0, 'N/R': 0},
                {'Team': team2, 'W': 1, 'L': 0, 'T': 0, 'N/R': 0},
            ])
    else:
        team1 = group.iloc[0]['Batting Team']
        team2 = group.iloc[0]['Bowling Team']
        match_results.extend([
            {'Team': team1, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1},
            {'Team': team2, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1},
        ])

results_df = pd.DataFrame(match_results)
summary_stats = results_df.groupby('Team').sum().reset_index()
summary_stats['M'] = summary_stats[['W', 'L', 'T', 'N/R']].sum(axis=1)
summary_stats['PT'] = summary_stats['W'] * 4 + summary_stats['T'] * 2 + summary_stats['N/R'] * 2

# -----------------------------
# Merge with NRR Table
# -----------------------------
final_merged = pd.merge(summary_stats, final_nrr_table, on='Team', how='outer')

# Bonus points input
bonus_points = {
    'Middlesex Women': 1,
    'Yorkshire Women': 2,
    'Derbyshire Falcons Women': 1
}
final_merged['BP'] = final_merged['Team'].map(bonus_points).fillna(0).astype(int)
final_merged['PT'] += final_merged['BP']

# Final formatted table
final_display = final_merged[
    ['Team', 'M', 'W', 'L', 'T', 'N/R', 'BP', 'PT', 'NRR',
     'Runs For', 'Overs For', 'Runs Against', 'Overs Against']
].sort_values(by=['PT', 'NRR'], ascending=[False, False]).reset_index(drop=True)

final_display.index += 1

# Define groupings
north_group = ['Yorkshire Women', 'Northamptonshire Steelbacks Women', 'Derbyshire Falcons Women', 'Leicestershire Foxes Women', 'Worcestershire Rapids Women']
south_group = final_display[~final_display['Team'].isin(north_group)]['Team'].tolist()

# Split into two tables
north_table = final_display[final_display['Team'].isin(north_group)].reset_index(drop=True)
south_table = final_display[final_display['Team'].isin(south_group)].reset_index(drop=True)

# Add rank index starting from 1
north_table.index += 1
south_table.index += 1

north_table

,Team,M,W,L,T,N/R,BP,PT,NRR,Runs For,Overs For,Runs Against,Overs Against
1,Yorkshire Women,2,2,0,0,0,2,10,1.977,232,32.4,205,40.0
2,Northamptonshire Steelbacks Women,2,2,0,0,0,0,8,0.575,257,40.0,234,40.0
3,Derbyshire Falcons Women,2,1,1,0,0,1,5,-0.505,161,37.1,158,32.4
4,Worcestershire Rapids Women,2,0,2,0,0,0,0,-0.839,172,40.0,191,37.1
5,Leicestershire Foxes Women,2,0,2,0,0,0,0,-0.850,216,40.0,250,40.0


In [17]:
south_table

,Team,M,W,L,T,N/R,BP,PT,NRR,Runs For,Overs For,Runs Against,Overs Against
1,Middlesex Women,3,3,0,0,0,1,13,0.820,322,55.1,301,60.0
2,Glamorgan Women,2,1,1,0,0,0,4,0.053,210,40.0,207,39.5
3,Sussex Sharks Women,3,1,2,0,0,0,4,-0.152,378,60.0,357,55.2
4,Kent Women,1,0,1,0,0,0,0,-0.900,122,20.0,140,20.0
5,Gloucestershire Women,1,0,1,0,0,0,0,-1.350,121,20.0,148,20.0
